In [1]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.preprocessing import LabelEncoder
import pandas as pd
import xgboost as xgb
import joblib
import numpy as np

In [2]:
# Load the updated dataset
url_dataset = pd.read_csv("../split_urls.csv")
url_dataset

,category,protocol,domain,path,query,fragment
0,malicious,NaN,NaN,br-icloud.com.br,NaN,NaN
1,benign,NaN,NaN,mp3raid.com/music/krizz_kaliko.html,NaN,NaN
2,benign,NaN,NaN,bopsecrets.org/rexroth/cr/1.htm,NaN,NaN
3,malicious,http,www.garage-pirenne.be,/index.php,option=com_content&view=article&id=70&vsig70_0=15,NaN
4,malicious,http,adventure-nicaragua.net,/index.php,option=com_mailto&tmpl=component&link=aHR0cDov...,NaN
...,...,...,...,...,...,...
651186,malicious,NaN,NaN,xbox360.ign.com/objects/850/850402.html,NaN,NaN
651187,malicious,NaN,NaN,games.teamxbox.com/xbox-360/1860/Dead-Space/,NaN,NaN
651188,malicious,NaN,NaN,www.gamespot.com/xbox360/action/deadspace/,NaN,NaN
651189,malicious,NaN,NaN,en.wikipedia.org/wiki/Dead_Space_(video_game),NaN,NaN


In [3]:
# Replace empty cells with whitespace
url_dataset = url_dataset.fillna("")

In [4]:
# Extract URLs and labels from the dataset
all_urls = url_dataset['protocol'] + "://" + url_dataset['domain'] + url_dataset['path'] + url_dataset['query'] + url_dataset['fragment']
labels = np.array((url_dataset['category'] == 'malicious').astype(int)).reshape(-1, 1)

In [5]:
# Use LabelEncoder to convert string labels to numerical format
label_encoder = LabelEncoder()
labels = label_encoder.fit_transform(labels)

d:\Users\redac\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\preprocessing\_label.py:114: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [6]:
# Create a TfidfVectorizer for feature extraction
vectorizer = TfidfVectorizer(binary=True)  # You can use TfidfVectorizer as well
X = vectorizer.fit_transform(all_urls)

In [7]:
# Split the dataset into a training set and a testing set
X_train, X_test, y_train, y_test = train_test_split(X, labels, test_size=0.2, random_state=42)

In [8]:
# Train an XGBoost classifier
xgb_classifier = xgb.XGBClassifier(random_state=42)
xgb_classifier.fit(X_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, random_state=42, ...)

In [9]:
# Predict labels for the test set
y_pred = xgb_classifier.predict(X_test)

In [10]:
# Calculate evaluation metrics
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average="weighted")
recall = recall_score(y_test, y_pred, average="weighted")
f1 = f1_score(y_test, y_pred, average="weighted")

In [11]:
# Display evaluation metrics
print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1-Score: {f1}")

Accuracy: 0.9421601824338331
Precision: 0.9430782012286097
Recall: 0.9421601824338331
F1-Score: 0.9413768654336122


In [ ]:
# Save the trained model and Vectorizer to files
model_filename = "Models/cv_xgb.pkl"
vectorizer_filename = "Vectorizers/vectorizer_cv_xgb.pkl"
joblib.dump(xgb_classifier, model_filename)
joblib.dump(vectorizer, vectorizer_filename)
label_encoder_filename = "Label Encoders/label_encoder_cv_xgb.pkl"
joblib.dump(label_encoder, label_encoder_filename)